# Evaluate Generated Captions

In [16]:
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 1.4 MB/s eta 0:00:001.6 MB/s eta 0:00:01


In [69]:
import json

def load_captions(filename, model):
    references = []
    predictions = []
    for i, line in enumerate(open(filename, "r").readlines()):
        caption_dict = json.loads(line)
        references.append([caption_dict["caption_0"]])
        references.append([caption_dict["caption_1"]])
        predictions.append(caption_dict[f"{model}_caption_0"])
        predictions.append(caption_dict[f"{model}_caption_1"])
    return references, predictions

In [70]:
models = [
    (path + "ofa_tiny.jsonl", "OFA$_{Tiny}$", 20, 20, "single-stream", "merged"),
    (path + "ofa_base.jsonl", "OFA$_{Base}$", 20, 20, "single-stream", "merged"),
    (path + "ofa_medium.jsonl", "OFA$_{Medium}$", 20, 20, "single-stream", "merged"),
    (path + "ofa_large.jsonl", "OFA$_{Large}$", 20, 20, "single-stream", "merged"),
    (path + "ofa_huge.jsonl", "OFA$_{Huge}$", 20, 20, "single-stream", "merged"),
    (path + "blip_b16.jsonl", "BLIP (ViT-B/16)", 129, 130, "dual-stream", "modality-specific, merged"),
    (path + "blip_l16.jsonl", "BLIP (ViT-L/16)", 129, 130, "dual-stream", "modality-specific"),
]

In [72]:
from collections import defaultdict
import evaluate

bleu = evaluate.load("bleu")
scores = defaultdict(dict)

for model in models:
    name = "blip" if "blip" in model[0] else "ofa"
    references, predictions = load_captions(model[0], name)
    for i in range(1, 5):
        results = bleu.compute(predictions=predictions, references=references, max_order=i)
        scores[model[1]][f"bleu-{i}"] = round(results["bleu"]*100, 2)

In [73]:
sacrebleu = evaluate.load("sacrebleu")

for model in models:
    name = "blip" if "blip" in model[0] else "ofa"
    references, predictions = load_captions(model[0], name)
    results = sacrebleu.compute(predictions=predictions, references=references)
    scores[model[1]]["sacrebleu"] = round(results["score"], 2)

In [75]:
import pandas as pd

scores_df = pd.DataFrame(scores).transpose()
scores_df

,bleu-1,bleu-2,bleu-3,bleu-4,sacrebleu
OFA$_{Tiny}$,14.40,5.76,2.50,1.30,1.30
OFA$_{Base}$,16.68,7.12,3.26,1.58,1.58
OFA$_{Medium}$,16.28,6.47,2.84,1.39,1.39
OFA$_{Large}$,15.10,6.45,3.03,1.53,1.53
OFA$_{Huge}$,15.73,6.94,3.06,1.35,1.35
BLIP (ViT-B/16),17.80,8.10,3.96,2.01,2.01
BLIP (ViT-L/16),17.96,8.31,4.36,2.50,2.50
